In [48]:
#Import Dependencies
import os
import pandas as pd

In [49]:
#Define pathways for datafiles
path_schools = os.path.join("Resources", "schools_complete.csv")
path_students = os.path.join("Resources", "students_complete.csv")

#Load in datafiles
schools_df = pd.read_csv(path_schools)
students_df = pd.read_csv(path_students)

#Merge data files bases on school
school_data_complete = pd.merge(students_df, schools_df, how="left", on=["school_name", "school_name"])


## Store Basic info for District

In [50]:
#Create snapshot of the district's key metrics
district_num_schools = len(school_data_complete["school_name"].unique())    #Store number of schools
district_num_students = len(school_data_complete["Student ID"].unique())  #Store number of students

district_by_school = school_data_complete.drop_duplicates(subset = ["school_name"]) #Made a DataFrame where there are not duplicates of schools (one student shown for each school)
district_total_budget = district_by_school["budget"].sum()  #Store total budget for district

# Distict Summary Stats

In [75]:
#Average scores
district_math_avg = school_data_complete["math_score"].mean()
district_reading_avg = school_data_complete["reading_score"].mean()

bins = [0, 69, 100] #Bins for failing (below 70) versus passing with max of 100
labels = ["fail", "pass"]   #Two bins: fail or pass
school_data_complete["pass/fail_reading"] = pd.cut(school_data_complete["reading_score"], bins, labels = labels, include_lowest=False) #Make new column for pass or failing reading
school_data_complete["pass/fail_math"] = pd.cut(school_data_complete["math_score"], bins, labels = labels, include_lowest=False)     #Make new column for pass or failing math

district_math_count = school_data_complete.groupby(["pass/fail_math"]).count()["Student ID"]    #Make series of students who fail and pass math
district_math_perc = district_math_count[1]/(district_math_count.sum())*100     #Calculate passing rate for math

district_reading_count = school_data_complete.groupby(["pass/fail_reading"]).count()["Student ID"]    #Make series of students who fail and pass reading
district_reading_perc = district_reading_count[1]/float(district_reading_count.sum())*100    #Calculate passing rate for reading

#Calculate how mamy students passed reading AND math
#-->Check to see if student's passed in both columns by creating df with criteria for "pass"
district_pass_both = school_data_complete.loc[(school_data_complete["pass/fail_math"] == "pass") & (school_data_complete["pass/fail_reading"] == "pass"), ["student_name", "pass/fail_math", "pass/fail_reading"]]
district_pass_both_count = district_pass_both["student_name"].count()   #Get the number of students in new df
district_pass_both_perc = district_pass_both_count/district_num_students*100   #Calculate % passing by deviding by the number of total students


In [76]:

#Create a Data Frame to Hold District Information
district_summary_df = pd.DataFrame({
    "Total Schools": [district_num_schools],
    "Total Students": [district_num_students],
    "Total Budget": [district_total_budget],
    "Average Math Score": [district_math_avg],
    "Average Reading Score": [district_reading_avg],
    "% Passing Math": [district_math_perc],
    "% Passing Reading": [district_reading_perc],
    "% Overall Passing": [district_pass_both_perc]
})

#Clean up formatting
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:,.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:,.2f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:,.2f}%".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:,.2f}%".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:,.2f}%".format)

pd.DataFrame(district_summary_df)

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,65.17%


# School's Summaries

In [55]:
#School Basic info

gby_schools = school_data_complete.groupby(["school_name"]) #Create Groupby object organized by the school

district_by_school_sorted = district_by_school.sort_values("school_name")

school_names = (district_by_school_sorted["school_name"]).values.tolist() #Make a list of school names
school_types = (district_by_school_sorted["type"]).values.tolist()   #make a list of school types that have the same indicies as the lit of school names
school_num_students = (gby_schools["student_name"].count()).values.tolist() #Make a list of the total the number of students for each school
school_budgets = (district_by_school_sorted["budget"]).values.tolist()     #make a list of budgets that have the same indicies as the list of schoosl


In [56]:
#Calculations for per student budget
school_budget_per_student = []
for i in range(district_num_schools):  #Loop through the number of schools
    school_budget_per_student.append((school_budgets[i])/(school_num_students[i]))  #Calculate the budget per student and add to list 


In [57]:
#Calculations for scores 
avg_math_scores = (gby_schools["math_score"].mean()).values.tolist()
avg_read_scores = (gby_schools["reading_score"].mean()).values.tolist()

#############################################################################

#Create df for kids who only passed math. Include columns for school_name and Student ID
passing_math_df = school_data_complete.loc[school_data_complete["pass/fail_math"] == "pass", ["school_name", "Student ID"]]
schools_passing_math_counts = (passing_math_df.groupby("school_name")["Student ID"].count()).values.tolist()    #Groupby school

#Loop through list of school and make a list with the percet of students who passed math
schools_pass_math_perc = []
for i in range(district_num_schools):
    schools_pass_math_perc.append((schools_passing_math_counts[i]/school_num_students[i])*100)

#############################################################################

#Create df for kids who only passed reading. Include columns for school_name and Student ID
passing_reading_df = school_data_complete.loc[school_data_complete["pass/fail_reading"] == "pass", ["school_name", "Student ID"]]
schools_passing_reading_counts = (passing_reading_df.groupby("school_name")["Student ID"].count()).values.tolist()

#Loop through list of school and make a list with the percet of students who passed reading
schools_pass_reading_perc = []
for i in range(district_num_schools):
    schools_pass_reading_perc.append((schools_passing_reading_counts[i]/school_num_students[i])*100)

In [58]:
#Percent of kids who passed both reading AND math at each school
#Create df of kids who passed both reading AND math
schools_pass_df = school_data_complete.loc[(school_data_complete["pass/fail_math"] == "pass") & (school_data_complete["pass/fail_reading"] == "pass"), ["school_name", "Student ID"]]
schools_pass_counts = schools_pass_df.groupby("school_name")["Student ID"].count()  #Groupby school and count number of ids

#Loop through schools and calculate percent at each school
schools_pass_both = []
for i in range(district_num_schools):
    schools_pass_both.append((schools_pass_counts[i])/(school_num_students[i])*100)



In [60]:
#Create Data Frame with Summaries for each school.

school_summary_df = pd.DataFrame({
    "School Name": school_names,
    "School Type": school_types,
    "Total Students": school_num_students,
    "Total Budget": school_budgets,
    "Per Student Budget": school_budget_per_student,
    "Average Math Score": avg_math_scores,
    "Average Reading Score": avg_read_scores,
    "% Passing Math": schools_pass_math_perc,
    "% Passing Reading": schools_pass_reading_perc,
    "% Overall Passing": schools_pass_both
})

#Make an original copy to use later (before formatting):
school_summary_og_df = pd.DataFrame({
    "School Name": school_names,
    "School Type": school_types,
    "Total Students": school_num_students,
    "Total Budget": school_budgets,
    "Per Student Budget": school_budget_per_student,
    "Average Math Score": avg_math_scores,
    "Average Reading Score": avg_read_scores,
    "% Passing Math": schools_pass_math_perc,
    "% Passing Reading": schools_pass_reading_perc,
    "% Overall Passing": schools_pass_both
})

#Clean Up Formatting 
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Total Budget"] = school_summary_df["Total Budget"].map("${:,}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("${:,}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["% Passing Math"] = school_summary_df["% Passing Math"].map("{:,.2f}%".format)
school_summary_df["% Passing Reading"] = school_summary_df["% Passing Reading"].map("{:,.2f}%".format)
school_summary_df["% Overall Passing"] = school_summary_df["% Overall Passing"].map("{:,.2f}%".format)

school_summary_df

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"4,976","$3,124,928",$628.0,77.05,81.03,66.68%,81.93%,54.64%
1,Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06,83.98,94.13%,97.04%,91.33%
2,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71,81.16,65.99%,80.74%,53.20%
3,Ford High School,District,"2,739","$1,763,916",$644.0,77.10,80.75,68.31%,79.30%,54.29%
4,Griffin High School,Charter,"1,468","$917,500",$625.0,83.35,83.82,93.39%,97.14%,90.60%
5,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29,80.93,66.75%,80.86%,53.53%
6,Holden High School,Charter,427,"$248,087",$581.0,83.80,83.81,92.51%,96.25%,89.23%
7,Huang High School,District,"2,917","$1,910,635",$655.0,76.63,81.18,65.68%,81.32%,53.51%
8,Johnson High School,District,"4,761","$3,094,650",$650.0,77.07,80.97,66.06%,81.22%,53.54%
9,Pena High School,Charter,962,"$585,858",$609.0,83.84,84.04,94.59%,95.95%,90.54%


# Highest Performing Schools by % Overall Passing

In [74]:
#Sort school summary df by % overall passing
highest_performing_df = school_summary_df.sort_values("% Overall Passing", ascending= False)

#Cut the highest performing df to the top 5
highest_performing_df = highest_performing_df.iloc[:5, :]

highest_performing_df = highest_performing_df.style.hide(axis='index')    #Hide the index! 

highest_performing_df

School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$1,043,130",$638.0,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$917,500",$625.0,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858",$609.0,83.84,84.04,94.59%,95.95%,90.54%


# Lowest Performing Schools by % Overall Passing

In [73]:
#Sort school summary df by % overall passing
lowest_performing_df = school_summary_df.sort_values("% Overall Passing")

#Cut the lowest performing df to the top 5
lowest_performing_df = lowest_performing_df.iloc[:5, :]

lowest_performing_df = lowest_performing_df.style.hide(axis='index')  #Hide the index! 

lowest_performing_df

School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$1,910,635",$655.0,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$3,094,650",$650.0,77.07,80.97,66.06%,81.22%,53.54%


# Math Scores by Grade

In [63]:
math_df = school_data_complete.groupby(["school_name", "grade"])[["math_score"]].mean() #Make a df where the scores for each school are grouped by grade

#Clean up formatting
math_df["math_score"] = math_df["math_score"].map("{:,.2f}".format)

mathscoresbygrade_df = pd.pivot_table(math_df, index='school_name', columns='grade', #Turn df into pivot table in order to change the axis in which the secondary index is on
                       aggfunc='mean', fill_value=0)

mathscoresbygrade_f_df = (mathscoresbygrade_df.swaplevel(0,1, axis=1).sort_index(1)
                                 .reindex(['9th','10th','11th','12th'], level=0, axis=1))       #Need to reorder the columns, so using Swaplevel since this is a multiindex table
mathscoresbygrade_f_df


/var/folders/1w/6l663v0951s50_1bzk8c5wt00000gp/T/ipykernel_25693/3335229058.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  mathscoresbygrade_f_df = (mathscoresbygrade_df.swaplevel(0,1, axis=1).sort_index(1)


grade,9th,10th,11th,12th
,math_score,math_score,math_score,math_score
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23


# Reading Scores by Grade

In [64]:
reading_df = school_data_complete.groupby(["school_name", "grade"])[["reading_score"]].mean()   #Make a df where the scores for each school are grouped by grade

#Clean up formatting
reading_df["reading_score"] = reading_df["reading_score"].map("{:,.2f}".format) 

readingscoresbygrade_df = pd.pivot_table(reading_df, index='school_name', columns='grade',  #Turn df into pivot table in order to change the axis in which the secondary index is on
                        aggfunc='mean', fill_value=0)

readingscoresbygrade_f_df = (readingscoresbygrade_df.swaplevel(0,1, axis=1).sort_index(1)
                                 .reindex(['9th','10th','11th','12th'], level=0, axis=1))       #Need to reorder the columns, so using Swaplevel since this is a multiindex table
readingscoresbygrade_f_df


/var/folders/1w/6l663v0951s50_1bzk8c5wt00000gp/T/ipykernel_25693/3016807488.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  readingscoresbygrade_f_df = (readingscoresbygrade_df.swaplevel(0,1, axis=1).sort_index(1)


grade,9th,10th,11th,12th
,reading_score,reading_score,reading_score,reading_score
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31


# Scores by Schools spending    

In [65]:
bins_spending = [0, 585, 630, 645, 680 ] #Bins with $20 range
labels_spending = ["<585", "585-630", "630-645", "645-680"]   #Four bins 
spending_classifications = pd.cut(school_summary_og_df["Per Student Budget"], bins_spending, labels = labels_spending, include_lowest=True) #Make list of spending classifications for each school

school_summary_spending_df = school_summary_og_df   #Make new df copy
school_summary_spending_df["Spending Classifications"] = spending_classifications   #Make a column for spending classification 
#Make dataframe grouping by spending classifications and include the required columns
spending_df = school_summary_spending_df.groupby("Spending Classifications")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()    

#Formatting
spending_df["Average Math Score"] = spending_df["Average Math Score"].map("{:,.2f}".format)
spending_df["Average Reading Score"] = spending_df["Average Reading Score"].map("{:,.2f}".format)
spending_df["% Passing Math"] = spending_df["% Passing Math"].map("{:,.2f}%".format)
spending_df["% Passing Reading"] = spending_df["% Passing Reading"].map("{:,.2f}%".format)
spending_df["% Overall Passing"] = spending_df["% Overall Passing"].map("{:,.2f}%".format)


spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Classifications,,,,,
<585,83.46,83.93,93.46%,96.61%,90.37%
585-630,81.90,83.16,87.13%,92.72%,81.42%
630-645,78.52,81.62,73.48%,84.39%,62.86%
645-680,77.00,81.03,66.16%,81.13%,53.53%


# Scores by School Size

In [66]:
bins_size = [0, 1000, 2000, 5000] #Bins for school size (pop)
labels_size = ["Small (<1000)", "Medium (1000-2000", "Large (2000-3000"]   #Three bins 
school_size_classifications = pd.cut(school_summary_og_df["Total Students"], bins_size, labels = labels_size, include_lowest=True) #Make list of the school size class for each school

school_summary_size_df = school_summary_og_df
school_summary_size_df["Scores by School Size"] = school_size_classifications   #Add a column with the school size classifications
#Make a new df grouped by the school size and with the required columns
scorebysize_df = school_summary_size_df.groupby("Scores by School Size")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()

#Formatting
scorebysize_df["Average Math Score"] = scorebysize_df["Average Math Score"].map("{:,.2f}".format)
scorebysize_df["Average Reading Score"] = scorebysize_df["Average Reading Score"].map("{:,.2f}".format)
scorebysize_df["% Passing Math"] = scorebysize_df["% Passing Math"].map("{:,.2f}%".format)
scorebysize_df["% Passing Reading"] = scorebysize_df["% Passing Reading"].map("{:,.2f}%".format)
scorebysize_df["% Overall Passing"] = scorebysize_df["% Overall Passing"].map("{:,.2f}%".format)

scorebysize_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Scores by School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000,83.37,83.86,93.60%,96.79%,90.62%
Large (2000-3000,77.75,81.34,69.96%,82.77%,58.29%


# Scores by School Type 

In [67]:
#Make a df where the schools are grouped by the school type (charter versus district) and the required columns
scoresbytype_df = school_summary_og_df.groupby("School Type")[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]].mean()

#Formatting
scoresbytype_df["Average Math Score"] = scoresbytype_df["Average Math Score"].map("{:,.2f}".format)
scoresbytype_df["Average Reading Score"] = scoresbytype_df["Average Reading Score"].map("{:,.2f}".format)
scoresbytype_df["% Passing Math"] = scoresbytype_df["% Passing Math"].map("{:,.2f}%".format)
scoresbytype_df["% Passing Reading"] = scoresbytype_df["% Passing Reading"].map("{:,.2f}%".format)
scoresbytype_df["% Overall Passing"] = scoresbytype_df["% Overall Passing"].map("{:,.2f}%".format)

scoresbytype_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
